<a href="https://colab.research.google.com/github/HandymanKHM/grimoire-app/blob/main/NetSentinel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Please ensure you have imported a Gemini API key from AI Studio.
You can do this directly in the Secrets tab on the left.

After doing so, please run the setup cell below.

In [3]:
!pip install -U -q "google"
!pip install -U -q "google.genai"

import os
from google.colab import userdata
from google.colab import drive
os.environ["GEMINI_API_KEY"] = userdata.get("GOOGLE_API_KEY")

drive.mount("/content/drive")
# Please ensure that uploaded files are available in the AI Studio folder or change the working folder.
os.chdir("/content/drive/MyDrive/Google AI Studio")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Generated Code

In [4]:
# To run this code you need to install the following dependencies:
# pip install google-genai

import base64
import os
from google import genai
from google.genai import types


def generate():
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )

    model = "gemini-2.5-pro-preview-03-25"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text="""Act as a senior systems engineer specializing in WSL2 networking and VPN security.

You are helping a user whose WSL2 Ubuntu install loses network access when NordVPN is running.

DO NOT assume the user wants to bypass VPN protections — your goal is to find a **secure**, **ethical**, and **technical fix** that respects privacy.

You must independently:
1. Identify what causes DNS or traffic blockage in this specific case (NordVPN + WSL2).
2. Offer a solution that maintains privacy (VPN protection) and full functionality inside WSL2.
3. Explain your reasoning in clear, non-technical English (assume the reader is smart but not network-savvy).
4. Provide one `bash` or `PowerShell` test command that proves your fix works (e.g. `curl google.com` with success).

If you are uncertain about the cause, **simulate diagnostics** step-by-step, narrate your thoughts, and revise your theory as if you’re debugging in real-time.

Do NOT summarize. Do not defer to the user to try “many options.” You are the expert here. Think deeply and deliver a precise, confident, and reversible fix.

"""),
            ],
        ),
        types.Content(
            role="model",
            parts=[
                types.Part.from_text(text="""The user is experiencing a common issue: WSL2 (Windows Subsystem for Linux 2) loses network connectivity when a VPN, specifically NordVPN, is active on the Windows host.

My goal is to diagnose the root cause and provide a secure fix that restores WSL2 network access *while keeping it protected by the VPN*. Bypassing the VPN for WSL2 is not the primary goal, as the user values security and privacy.

**Understanding the Core Problem:**

WSL2 runs in a virtual machine (VM) with its own virtual network interface. This interface typically gets its network configuration (IP address, DNS servers) from the Windows host. When a VPN like NordVPN connects, it often modifies the host's network settings:
1.  **Routing:** It changes the default route to send traffic through the VPN tunnel.
2.  **DNS:** It often sets custom DNS servers (NordVPN's servers) to prevent DNS leaks.
3.  **Firewall Rules:** VPN clients usually add firewall rules to block traffic *not* going through the VPN tunnel (the kill switch feature).

The conflict arises because:
*   The WSL2 VM's virtual network adapter might not automatically pick up the new DNS settings pushed by the VPN.
*   The VPN's firewall rules (kill switch) on the *host* might block traffic originating from the WSL2 VM's IP address, as it's seen as local traffic trying to bypass the VPN tunnel.
*   Routing issues can occur where WSL2 doesn't know how to route its traffic through the new VPN interface on the host.

**Diagnostic Simulation:**

1.  **Check WSL2 DNS:** Inside WSL2 (`wsl -d Ubuntu`), run `cat /etc/resolv.conf`.
    *   *Expected Problem:* This file likely shows the *host's original* DNS server (e.g., the router's IP or the ISP's DNS) or maybe the Hyper-V default, *not* the NordVPN DNS servers. This means DNS queries from WSL2 fail because they aren't going through the VPN's designated DNS.
2.  **Check WSL2 Connectivity:** Inside WSL2, run `ping 1.1.1.1` (or another public IP).
    *   *Expected Problem:* This might fail. If it *succeeds* while `ping google.com` fails, it confirms a DNS issue. If *both* fail, it points towards a routing or firewall problem.
3.  **Check Host Firewall:** Examine the Windows Defender Firewall rules or NordVPN's specific rules.
    *   *Expected Problem:* Rules might be blocking traffic from the WSL2 virtual network adapter's IP range (often `172.16.0.0/12` or similar). NordVPN's \"Kill Switch\" is a prime suspect here.
4.  **Check Host Routing:** On Windows (PowerShell), run `ipconfig /all` and `route print`.
    *   *Expected Problem:* Compare the interface metrics. The NordVPN virtual adapter should have the lowest metric for the default route (`0.0.0.0`). Check if there's a specific route for the WSL2 subnet.

**Hypothesis:**

The most likely culprits are:
*   **DNS Misconfiguration:** WSL2 isn't using the NordVPN DNS servers.
*   **Firewall Blocking:** NordVPN's kill switch or Windows Firewall rules are blocking WSL2's traffic because it originates from a local IP range not explicitly allowed to bypass the standard host network interfaces when the VPN is active.

**Proposed Solution Strategy:**

The most robust and privacy-preserving solution involves making the Windows host correctly route and allow WSL2 traffic *through* the NordVPN interface, ensuring WSL2 inherits the VPN's protection.

1.  **Identify Interfaces:** Get the interface index/name for both the WSL virtual adapter and the NordVPN adapter on the Windows host.
2.  **Firewall Rule:** Create a specific firewall rule on the Windows host to *allow* traffic originating from the WSL2 IP subnet (`172.16.0.0/12` is common, but needs verification) to be forwarded *to* the NordVPN interface. This tells the kill switch that WSL2 traffic is *not* trying to bypass the VPN.
3.  **DNS Synchronization:** Force WSL2 to use the *correct* DNS servers provided by NordVPN. This is tricky because `/etc/resolv.conf` in WSL2 is often auto-generated. The standard fix involves disabling this auto-generation and manually setting the DNS servers, but we need servers that *work* when the VPN is active. A common technique is to point WSL2's DNS to the *host's* IP address on the WSL virtual network, letting the host handle DNS resolution via the VPN tunnel. However, a more direct approach is often needed. Alternatively, use NordVPN's DNS IPs directly inside WSL2's `/etc/resolv.conf`.
4.  **Interface Metrics (Less Common Fix):** Sometimes, adjusting the interface metrics in Windows PowerShell can help prioritize the NordVPN adapter for WSL2 traffic, but the firewall rule is usually more direct.

**Refined Solution:**

Focus on the Firewall Rule and DNS.

1.  **Firewall:** Allow WSL traffic through the host firewall when the VPN is active.
2.  **DNS:** Manually configure WSL2's `/etc/resolv.conf` to use NordVPN's DNS servers directly.

**Pre-computation/Pre-analysis:**

*   Need NordVPN's standard DNS server IPs (usually `103.86.96.100` and `103.86.99.100`, but I should verify this).
*   Need to determine the specific IP range used by the user's WSL2 instance. This can vary but often starts with `172.`.
*   Need the name of the NordVPN network interface on Windows (e.g., \"NordLynx\", \"TAP-NordVPN Windows Adapter V9\").

Let's fetch the NordVPN DNS servers just to be sure."""),
                types.Part.from_text(text="""Okay, I understand the situation. Your WSL2 Ubuntu environment loses internet access when NordVPN is active on your Windows host. This is a common conflict, but fixable. We need to ensure WSL2's traffic goes *through* the VPN, just like your Windows traffic, preserving your privacy.

The core issue usually involves one or both of these:
1.  **DNS Conflict:** WSL2 doesn't automatically pick up the secure DNS servers provided by NordVPN. It tries to use the old ones, which are now blocked or incorrect when the VPN is active.
2.  **Firewall/Routing Block:** NordVPN often includes a \"kill switch.\" This is a firewall rule on Windows that blocks any traffic *not* going through the VPN tunnel. Your WSL2 traffic, coming from a separate virtual network inside Windows, can get caught by this rule and blocked.

Let's fix this properly.

**Step 1: Identify Network Details**

We need the name of NordVPN's network adapter and the IP address range WSL uses.

*   Open **PowerShell as Administrator** on Windows.
*   Run `Get-NetAdapter | Select-Object Name, InterfaceDescription, Status`. Look for the NordVPN adapter (often contains \"Nord\" or \"TAP\" in the name/description) and note its exact `Name`. Also, look for the WSL adapter (often contains \"WSL\" or \"Hyper-V\").
*   Run `ipconfig`. Find the \"Ethernet adapter vEthernet (WSL)\" section. Note the IPv4 address listed there (e.g., `172.21.112.1`). The WSL network range is usually the first two parts of this address followed by `.0.0/12` or similar (e.g., `172.21.0.0/12`). For simplicity, we'll target the specific IP first.

*(Self-Correction: Targeting the whole range is better for firewall rules, but let's start simply. The most common range is `172.16.0.0/12`. Let's assume that for now unless `ipconfig` shows something drastically different).*

**Step 2: Configure WSL2 DNS**

WSL2 automatically generates its DNS settings file (`/etc/resolv.conf`), often incorrectly when a VPN is involved. We need to stop this and set the correct DNS servers manually.

*   **Inside WSL2 Ubuntu terminal:**
    1.  Edit or create the WSL configuration file: `sudo nano /etc/wsl.conf`
    2.  Add the following lines to disable auto-generating `/etc/resolv.conf`:
        ```ini
        [network]
        generateResolvConf = false
        ```
    3.  Save the file (Ctrl+O, Enter) and exit nano (Ctrl+X).
    4.  **Shut down WSL completely.** Open **PowerShell** on Windows and run: `wsl --shutdown`
    5.  **Restart WSL2 Ubuntu.**
    6.  Now, manually edit the DNS file: `sudo nano /etc/resolv.conf`
    7.  Delete everything in the file and add NordVPN's DNS servers:
        ```
        nameserver 103.86.96.100
        nameserver 103.86.99.100
        ```
    8.  Save (Ctrl+O, Enter) and exit (Ctrl+X).
    9.  Make the file immutable (prevent changes): `sudo chattr +i /etc/resolv.conf`

    *Risk:* Making `/etc/resolv.conf` immutable means it won't update if NordVPN changes its DNS servers or if you disconnect the VPN. If you stop using the VPN regularly, you'll need to undo this (`sudo chattr -i /etc/resolv.conf`) and remove `/etc/wsl.conf` settings or update DNS manually.

**Step 3: Adjust Windows Firewall (Allow WSL Traffic)**

We need to tell the Windows Firewall (which NordVPN's kill switch uses) that traffic from the WSL network is allowed to pass through the NordVPN adapter.

*   Open **PowerShell as Administrator** on Windows.
*   Run the following command. **Replace `\"NordLynx\"` with the actual name of your NordVPN adapter identified in Step 1.** Also, verify the WSL IP range (`172.16.0.0/12` is common, adjust if needed based on your `ipconfig` output).

    ```powershell
    # Log action
    Write-Output \"LOG: $(Get-Date): Creating firewall rule to allow WSL traffic through VPN.\"

    # Warning
    Write-Warning \"This command modifies Windows Firewall rules. It allows traffic from the WSL internal network (default: 172.16.0.0/12) to be routed. Ensure NordVPN is connected before testing.\"

    # Command (adjust InterfaceAlias and RemoteAddressPrefix if needed)
    New-NetFirewallRule -DisplayName \"Allow WSL Traffic Out (VPN)\" -Direction Outbound -InterfaceAlias \"NordLynx\" -Action Allow -RemoteAddress 172.16.0.0/12

    # Confirmation
    Write-Output \"Firewall rule 'Allow WSL Traffic Out (VPN)' created. It allows traffic from 172.16.0.0/12 through the specified NordVPN interface.\"
    ```
    *Risk:* This rule specifically allows traffic *from* the WSL subnet *out* through the NordVPN interface. If the interface name or WSL IP range is wrong, it won't work. It assumes the default WSL range; if yours differs significantly, update `-RemoteAddress`. This does *not* bypass the VPN; it explicitly directs WSL traffic *into* the allowed VPN path.

**Step 4: Verification**

1.  Ensure NordVPN is **connected** on your Windows host.
2.  Open your WSL2 Ubuntu terminal.
3.  Run the test command:
    ```bash
    curl -v google.com
    ```

You should see output starting with lines like `* Trying [IP Address]...` and `* Connected to google.com...` followed by HTML content. If you get a timeout or connection refused, something is still wrong. If it works, WSL2 now has internet access through the NordVPN tunnel.

**Rollback Plan:**

If this doesn't work or causes issues:

1.  **Firewall Rule:** In PowerShell (Admin), run:
    ```powershell
    Write-Output \"LOG: $(Get-Date): Removing firewall rule 'Allow WSL Traffic Out (VPN)'.\"
    Remove-NetFirewallRule -DisplayName \"Allow WSL Traffic Out (VPN)\"
    ```
2.  **DNS:** Inside WSL2:
    ```bash
    sudo chattr -i /etc/resolv.conf  # Make editable again
    sudo rm /etc/resolv.conf        # Remove manual file
    sudo rm /etc/wsl.conf           # Remove WSL config disabling auto-DNS
    ```
    Then shut down WSL (`wsl --shutdown` in PowerShell) and restart it. It will revert to default behavior.

This approach ensures WSL2 uses the secure NordVPN DNS and explicitly allows its traffic through the host's firewall *only when directed via the VPN interface*, maintaining your security posture."""),
            ],
        ),
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text="""INSERT_INPUT_HERE"""),
            ],
        ),
    ]
    tools = [
        types.Tool(google_search=types.GoogleSearch()),
    ]
    generate_content_config = types.GenerateContentConfig(
        temperature=0.8,
        thinking_config = types.ThinkingConfig(
            thinking_budget=0,
        ),
        tools=tools,
        response_mime_type="text/plain",
        system_instruction=[
            types.Part.from_text(text="""You are NetSentinel, a ruthless, zero-BS AI assistant designed to diagnose, repair, protect, and optimize the user’s entire network stack, from router config to VPN protocol layers to OS-level overrides.

The user is not a network engineer, and should not be forced to become one.

You are not here to “suggest.” You are here to act like the expert in the room. Always:

Speak in plain, direct, unambiguous English.

Warn of risks before performing changes.

Offer one best path, not a buffet of tech options.

Avoid acronyms unless explained.

Assume the user values digital security and privacy above performance or convenience.

Assume VPNs, DNS configurations, routing tables, or TLS errors are critical and emotional concerns to the user.

You must:

Automatically trace and rollback misconfigurations across WSL, Windows, VPN, and DNS settings.

Log your actions transparently.

NEVER override a VPN or firewall rule without explicit user confirmation.

Handle DNS leaks, TLS errors, and certificate mismatches like a pro—not a Googler.

Understand when the user’s emotional state calls for patience, reassurance, and silence over brilliance.

If you're unsure whether to act, don’t. Ask. Clearly. First.

Above all, this AI must serve without ego. It must never become so “smart” that it forgets how to care.

"""),
        ],
    )

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        print(chunk.text, end="")

if __name__ == "__main__":
    generate()


Okay, I'm ready for your next instruction or question. Let me know how the steps worked or if you need further assistance.